In [1]:
using DataFrames
using CSV
using XLSX
using Statistics
using Plots
using Dates
using GLM
using MLBase
using LinearAlgebra

In [22]:
prices = XLSX.readtable("data/zillow_data_download_april2020.xlsx", "Sales_median_price_city", "A:ER")
counts = XLSX.readtable("data/zillow_data_download_april2020.xlsx", "Sale_counts_city", "A:ER")

#Columns we care about
atr = 1:4 #regionID, city, state, size atributes
years = 39:147 #2011-2020
colIndex = [atr; years]

pricesDF = DataFrame(prices[1][colIndex], prices[2][colIndex])
pricesData = dropmissing(pricesDF)

countsDF = DataFrame(counts[1][colIndex], counts[2][colIndex])
countsData = dropmissing(countsDF) #So we don't have to re-read to get old data

,RegionID,RegionName,StateName,SizeRank,2011-01,2011-02,2011-03
,Any,Any,Any,Any,Any,Any,Any
1,6181,New York,New York,1,2821,2865,3303
2,12447,Los Angeles,California,2,1834,1723,2391
3,39051,Houston,Texas,3,1960,1762,2707
4,17426,Chicago,Illinois,4,2111,1792,2457
5,6915,San Antonio,Texas,5,962,960,1372
6,13271,Philadelphia,Pennsylvania,6,961,981,1334
7,40326,Phoenix,Arizona,7,2529,2585,3558
8,18959,Las Vegas,Nevada,8,2358,2574,3127
9,54296,San Diego,California,9,876,924,1157


In [166]:
#Make initial plots
analysisDF = DataFrame(cityName = String[], stateName = String[], slope = Float64[], inter = Float64[])

#Vectorize input dates
datesInput = collect(Date(2011, 1):Dates.Month(1):Date(2020, 1))

#Top N cities by population we are going to analyze
N = 100

for row = 1:N
    #Vectorize outputs
    countsOutput = collect(countsData[row, 5:end])
    pricesOutput = collect(pricesData[row, 5:end])
    
    #Get city and state name
    cityName = countsData[row, 2]
    stateName = countsData[row, 3]  
    loc = string(cityName, ", ", stateName)
    
    #Best fit
    time = 1:109
    priceFitDF = DataFrame(t = time, medPrice = pricesOutput)
    priceFit = lm(@formula(medPrice ~ t), priceFitDF)
    
    slope = round(GLM.coef(priceFit)[2], digits = 3)
    inter = round(GLM.coef(priceFit)[1], digits = 3)
    reg = predict(priceFit)   
    
    push!(analysisDF, [cityName, stateName, slope, inter])
    
    #Plot: units sold and best fit, median price and best fit
    plot(datesInput, countsOutput, 
        title = loc, xlabel = "Month", ylabel = "Homes Sold", legend = false)
    png(string("Plots/Homes-Sold/", string(cityName, "-Homes-Sold")))
        
    scatter(datesInput, pricesOutput, 
        title = loc, xlabel = "Month", ylabel = "Median Price (\$)", label = "Zillow Data")
    
    plot!(datesInput, reg, lw = 3, label = "Lin. Reg", legend = :bottomright, linestyle = :dash)
    annotate!(datesInput[1], maximum(pricesOutput), Plots.text(string("m = ", slope), 10, :black, :left))
    
    png(string("Plots/Median-Price/", string(cityName, "-Median-Price")))
end

In [16]:
function evalSin(X, A)
    a = A[1]
    b = A[2]
    c = A[3]
    d = A[4]
    
    return a.*sin.(b.*X.+c).+d
end

function evalJacob(X, A)
    a = A[1]
    b = A[2]
    c = A[3]
    d = A[4]
    J = []
    
    for x in X
        da = sin(b*x + c)
        db = a*x*cos(b*x+c)
        dc = a*cos(b*x+c)
        
        if isempty(J)
            J = [da db dc 1]
        else
            J = [J; da db dc 1]
        end
    end
    
    return J
end

function newton(data, a0)
    alp = 0.05
    tol = 1e-3
    count = 0
    
    a0 = a0'
    R = data.Y .- evalSin(data.X, a0)
    J = evalJacob(data.X, a0)
    
    pInv = inv(J'*J)*J'
    A = a0 .+ alp*pInv*R

    while norm(a0-A, 2) > tol
        a0 = A
        
        R = data.Y .- evalSin(data.X, a0)
        J = evalJacob(data.X, a0)
        pInv = inv(J'*J)*J'
        
        A = a0 .+ alp*pInv*R
        
        count = count + 1
        
        if count > 1e5
            print("Broke with count of >1e5")
            break
        end
    end
    
    return A
end

newton (generic function with 1 method)

In [21]:
#Testing
x = collect(-5:0.01:5)
y = 1.5.*sin.(x+0.5*rand(Float64, size(x))).+3

data = DataFrame(X = x, Y = y)
a0 = [0.5 0.5 1 1]

A = newton(data, a0)
plot(x, y, linecolor = :red, label = "Data")
plot!(x, evalSin(x, A), linecolor = :blue, label = "Newton-Gauss")
png("Lets go!!")